In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, SubsetRandomSampler
import numpy as np

# GPU 사용 가능 여부 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# 데이터셋 다운로드 및 전처리
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 레이블이 있는 데이터와 레이블이 없는 데이터 분리 (10% 레이블 사용)
labeled_indices = np.random.choice(len(train_dataset), size=int(0.1 * len(train_dataset)), replace=False)
unlabeled_indices = list(set(range(len(train_dataset))) - set(labeled_indices))

labeled_sampler = SubsetRandomSampler(labeled_indices)
unlabeled_sampler = SubsetRandomSampler(unlabeled_indices)

# DataLoader 설정 (GPU로 전송)
batch_size = 64
labeled_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=labeled_sampler, num_workers=2, pin_memory=True)
unlabeled_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=unlabeled_sampler, num_workers=2, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True)

# 간단한 CNN 모델 정의
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
    
    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = nn.functional.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# 모델 초기화 및 GPU 설정
model = SimpleCNN().to(device)

# 모델 학습 함수 정의
def train(model, labeled_loader, unlabeled_loader, optimizer, criterion, epochs, patience=3):
    best_loss = float('inf')
    no_improvement = 0
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0.0
        for inputs, labels in labeled_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_loss = total_loss / len(labeled_loader)
        print(f'Epoch [{epoch + 1}/{epochs}], Avg Loss: {avg_loss:.4f}')

        # Early stopping
        if avg_loss < best_loss:
            best_loss = avg_loss
            no_improvement = 0
        else:
            no_improvement += 1
            if no_improvement >= patience:
                print(f'Early stopping! No improvement for {patience} epochs.')
                break

# 모델 및 최적화 함수 설정
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

# 초기 레이블이 있는 데이터로 모델 학습
train(model, labeled_loader, unlabeled_loader, optimizer, criterion, epochs=20, patience=3)

# 모델 성능 평가 함수 정의
def test(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

# 테스트 데이터로 모델 평가
test(model, test_loader)
